In [3]:
# mount the google drive path
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/Colab Notebooks/face-classification

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/face-classification


In [4]:
!pip install cmake face_recognition opencv-python

In [5]:
import os
import cv2
import pdb
import shutil
import encodings 
import face_recognition
from pathlib import Path
from google.colab.patches import cv2_imshow

In [6]:
IMAGE_DIR = './images'
RESULT_DIR = './results'
CROPPED_DIR = './cropped'
TOLERANE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = 'cnn'

faces = []
face_names = []
face_count = 0

In [7]:
def addUnknownFaceEncoding(encoding):
  global face_count
  match = f"face_{face_count:03d}"
  face_count+=1
  faces.append(encoding)
  face_names.append(match)
  return (f"{RESULT_DIR}/{match}", match)

def highLightImgWithFace(image, location, match):
  top_left_ = (location[3], location[0])
  bottom_right_ = (location[1], location[2])
  cv2.rectangle(image, top_left_, bottom_right_, [0,255,0], FRAME_THICKNESS)

  top_left = (location[3], location[2])
  bottom_right = (location[1], location[2]+22)
  cv2.rectangle(image, top_left, bottom_right, [0,255,0], cv2.FILLED)
  cv2.putText(image, match, (location[3]+10, location[2]+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), FONT_THICKNESS)
  return (image, top_left_, bottom_right_)

In [ ]:
print("processing images...")
# loop over the image paths
for filename in os.listdir(IMAGE_DIR):
    pdb.set_trace
    try:
      os.rmdir(RESULT_DIR)
      os.rmdir(CROPPED_DIR)
    except OSError as error:
      pass
    print(f"\n\nin file: {filename}\n=================")
    src_path=f"{IMAGE_DIR}/{filename}"
    # image = face_recognition.load_image_file(src_path)
    image = cv2.imread(src_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Use Face_recognition to locate faces
    face_locations = face_recognition.face_locations(image, model=MODEL)
    print(f"detected faces: {len(face_locations)}")
    face_encodings = face_recognition.face_encodings(image, face_locations)
    for i, (encoding, location) in enumerate(zip(face_encodings, face_locations)):
        results = face_recognition.compare_faces(faces, encoding, TOLERANE)
        match = None
        dst_path  = ''
        print(f"{i} encoding: {location}\t results: {results}")
        if(not results):
          dst_path, match = addUnknownFaceEncoding(encoding)
        else:
          for index, result in enumerate(results):
            if result:
                  match = face_names[index]
                  dst_path = f"{RESULT_DIR}/{match}"
                  print(f"Match found for {filename}: {match}")
            else:
                  dst_path, match = addUnknownFaceEncoding(encoding)
            # put box on face
            image, (x,y), (w,h) = highLightImgWithFace(image, location, match)
            # put detected face in repective folder
            Path(f"{CROPPED_DIR}").mkdir(parents=True, exist_ok=True)
            cropped_image = image[y:h, x:w]
            cv2.imwrite(f"{CROPPED_DIR}/{match}_e{i}_r{index}.jpg", cropped_image)
            
        # Check if director presented for current face
        Path(dst_path).mkdir(parents=True, exist_ok=True)
        shutil.copy2(src_path, dst_path)
          
    
    cv2_imshow(image)
    # cv2.waitKey(10000)
    # cv2.destroyWindow(filename)

In [10]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [9]:
!git config --global user.email "zinjadsaurabh1997@gmail.com"
!git config --global user.name "Saurabh Zinjad"
!git add .
!git commit -am "Modified code for move photo into classification folder"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
